## 🔧 Persiapan dan Impor Library

Langkah pertama adalah mempersiapkan lingkungan dengan mengimpor semua library yang diperlukan. Ini termasuk library untuk manipulasi file (`os`, `shutil`, `Path`), pemrosesan gambar (`PIL`, `matplotlib`, `ImageDraw`), serta pemrosesan anotasi XML dan dataset COCO (`pylabel`, `ET`, `minidom`).

Selain itu, digunakan `random` untuk konsistensi hasil eksperimen dan `tqdm` untuk progress bar saat iterasi.


In [1]:
import torch
import os 
import random
import shutil
from pathlib import Path
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import logging
import os 
import zipfile
import yaml
import PIL

from IPython.display import Image  # for displaying images
from sklearn.model_selection import train_test_split
from pylabel import importer
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw

logging.getLogger().setLevel(logging.CRITICAL)
random.seed(42)

## 📁 Mengecek Direktori Kerja dan Dataset

Sebelum mulai memproses data, kita perlu memastikan bahwa direktori kerja berada di lokasi yang benar, dan dataset yang akan digunakan sudah tersedia.

Dataset yang digunakan adalah **"Bottle Images"**, yang berisi gambar serta anotasi botol dari berbagai kategori. Namun, dalam proyek ini kita hanya akan mendeteksi keberadaan botol, tanpa membedakan jenisnya.


In [2]:
# Cek posisi direktori kerja saat ini
current_dir = os.getcwd()
print("Current working directory:", current_dir)

# List semua file & folder di direktori saat ini
items = os.listdir(current_dir)
print("/nIsi folder saat ini:")
for item in items:
    print("-", item)

DATASET_PATH = Path('./Bottle Images')
print("\nDataset path yang digunakan:", DATASET_PATH)

Current working directory: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle
/nIsi folder saat ini:
- .ipynb_checkpoints
- Bottle Images
- ImageClassesCombinedWithCOCOAnnotations
- YOLO.ipynb

Dataset path yang digunakan: Bottle Images


## 📊 Memuat Dataset dengan Format COCO

Dataset anotasi yang digunakan berada dalam format COCO (`coco_instances.json`). Untuk memuatnya, digunakan pustaka `pylabel` yang secara otomatis membaca file JSON COCO dan mengubahnya ke dalam bentuk `DataFrame` agar lebih mudah diproses.

- `path_to_annotations`: Lokasi file COCO JSON.
- `path_to_images`: Lokasi gambar yang sesuai dengan anotasi.

Hasil pemuatan akan menampilkan 5 baris pertama dari anotasi dalam format tabel. Meskipun setiap botol punya label seperti `plastic` atau `wine`, untuk tujuan proyek ini semua label akan disamakan menjadi satu kelas yaitu `bottle`.


In [3]:
#Specify path to the coco.json file
path_to_annotations = r"./ImageClassesCombinedWithCOCOAnnotations/coco_instances.json"
#Specify the path to the images (if they are in a different folder than the annotations)
path_to_images = r"./Bottle Images/images_raw"

#Import the dataset into the pylable schema 
dataset = importer.ImportCoco(path_to_annotations, path_to_images=path_to_images, name="BCCD_coco")
dataset.df.head(5)

,img_folder,img_filename,img_path,img_id,img_width,img_height,img_depth,ann_segmented,ann_bbox_xmin,ann_bbox_ymin,...,ann_iscrowd,ann_keypoints,ann_pose,ann_truncated,ann_difficult,cat_id,cat_name,cat_supercategory,split,annotated
id,,,,,,,,,,,,,,,,,,,,,
0,./Bottle Images/images_raw,00000000.jpg,,0,512,512,,,370.5,214.5,...,0,,,,,5,plastic,bottle_categories,,1
1,./Bottle Images/images_raw,00000000.jpg,,0,512,512,,,348.5,147.5,...,0,,,,,1,wine,bottle_categories,,1
2,./Bottle Images/images_raw,00000001.jpg,,1,512,512,,,84.5,100.5,...,0,,,,,5,plastic,bottle_categories,,1
3,./Bottle Images/images_raw,00000001.jpg,,1,512,512,,,263.5,172.5,...,0,,,,,1,wine,bottle_categories,,1
4,./Bottle Images/images_raw,00000001.jpg,,1,512,512,,,309.5,206.5,...,0,,,,,5,plastic,bottle_categories,,1


In [4]:
print(f"Number of images: {dataset.analyze.num_images}")
print(f"Number of classes: {dataset.analyze.num_classes}")
print(f"Classes:{dataset.analyze.classes}")
print(f"Class counts:\n{dataset.analyze.class_counts}")
print(f"Path to annotations:\n{dataset.path_to_annotations}")

Number of images: 15000
Number of classes: 5
Classes:['wine', 'water', 'beer', 'soda', 'plastic']
Class counts:
cat_name
plastic    6050
beer       6047
wine       5990
soda       5946
water      5940
Name: count, dtype: int64
Path to annotations:
ImageClassesCombinedWithCOCOAnnotations


In [12]:
import json
import os
from tqdm import tqdm

# === Konfigurasi ===
coco_json_path = "./ImageClassesCombinedWithCOCOAnnotations/coco_instances.json"
output_labels_dir = "./yolo_labels"
image_dir = "./Bottle Images/images_raw"  # folder gambar untuk referensi dimensi

# Buat folder label jika belum ada
os.makedirs(output_labels_dir, exist_ok=True)

# Load COCO JSON
with open(coco_json_path, 'r') as f:
    coco = json.load(f)

images = {img['id']: img for img in coco['images']}
annotations = coco['annotations']

print(f"Total annotations: {len(annotations)}")
print(f"Total unique images: {len(images)}")

# Simpan catatan file yang sudah dibuat
generated_files = set()

# Proses semua anotasi
for ann in tqdm(annotations):
    image_id = ann['image_id']
    img_info = images[image_id]
    width = img_info['width']
    height = img_info['height']
    image_filename = img_info['file_name']
    
    # Ambil bounding box COCO
    x, y, w, h = ann['bbox']

    # Konversi ke YOLO format (x_center, y_center, w, h) normalized
    x_center = (x + w / 2) / width
    y_center = (y + h / 2) / height
    w_norm = w / width
    h_norm = h / height

    # Semua class_id dianggap 0 (hanya 1 class: bottle)
    yolo_line = f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}"

    # Simpan dalam file .txt dengan nama sama seperti gambar
    txt_filename = os.path.splitext(image_filename)[0] + ".txt"
    txt_path = os.path.join(output_labels_dir, txt_filename)

    with open(txt_path, 'a') as f:
        f.write(yolo_line + "\n")

    generated_files.add(txt_filename)

# Verifikasi hasil
print(f"\n✅ Jumlah file .txt yang dihasilkan: {len(generated_files)}")
print(f"🗂️  Jumlah file unik gambar di COCO JSON: {len(images)}")
print(f"📁 Folder output: {output_labels_dir}")
print("\n📄 Contoh isi file label pertama:")

# Print isi contoh file
example_file = list(generated_files)[0]
with open(os.path.join(output_labels_dir, example_file), 'r') as f:
    lines = f.readlines()
    for line in lines:
        print("   ", line.strip())

Total annotations: 29973
Total unique images: 15000


100%|██████████████████████████████████████████████████████████████████████████| 29973/29973 [00:07<00:00, 3856.36it/s]


✅ Jumlah file .txt yang dihasilkan: 15000
🗂️  Jumlah file unik gambar di COCO JSON: 15000
📁 Folder output: ./yolo_labels

📄 Contoh isi file label pertama:
    0 0.423828 0.597656 0.169922 0.361328
    0 0.778320 0.137695 0.289062 0.226562
    0 0.423828 0.597656 0.169922 0.361328
    0 0.778320 0.137695 0.289062 0.226562


## 🔄 Menyamakan Semua Label Menjadi Satu Kelas: 'bottle'

Dataset aslinya memiliki beberapa kelas berbeda untuk jenis botol seperti `plastic`, `wine`, dan lain-lain. Namun, karena tujuan proyek ini adalah **mendeteksi keberadaan botol tanpa membedakan jenisnya**, maka semua label akan disamakan menjadi satu kelas umum yaitu `bottle`.

Langkah ini penting agar model tidak belajar membedakan jenis botol, tetapi fokus pada mendeteksi objek botol secara umum.

In [17]:
import pandas as pd
import json

# Paths
path_to_annotations = r"./ImageClassesCombinedWithCOCOAnnotations/coco_instances.json"
path_to_images = r"./Bottle Images/images_raw"

# Load COCO JSON
with open(path_to_annotations, 'r') as f:
    coco = json.load(f)

images = {img['id']: img for img in coco['images']}
annotations = coco['annotations']
categories = {cat['id']: cat['name'] for cat in coco['categories']}

# Buat list data untuk DataFrame
rows = []
for ann in annotations:
    img_info = images[ann['image_id']]
    cat_name = categories[ann['category_id']]
    # Simulasi class_id=0 karena gabungkan semua jadi 'bottle'
    class_id = 0
    row = {
        'img_folder': path_to_images,
        'img_filename': img_info['file_name'],
        'img_path': '',   # kosong sesuai contoh
        'img_id': ann['image_id'],
        'img_width': img_info['width'],
        'img_height': img_info['height'],
        'img_depth': '',  # bisa diisi jika ada info
        'ann_bbox_xmin': ann['bbox'][0],
        'ann_bbox_ymin': ann['bbox'][1],
        'ann_bbox_width': ann['bbox'][2],
        'ann_bbox_height': ann['bbox'][3],
        'cat_id': class_id,
        'cat_name': 'bottle',
        # Bisa tambah kolom lain kalau perlu
    }
    rows.append(row)

# Buat DataFrame
df = pd.DataFrame(rows)

# Tampilkan 5 baris pertama
print(df.head())

# Cek apakah semua class_id = 0
if (df['cat_id'] == 0).all():
    print("\n✅ All labels have class_id = 0 (bottle).")
else:
    print("\n⚠️ There are labels with class_id != 0.")

                   img_folder  img_filename img_path  img_id  img_width  \
0  ./Bottle Images/images_raw  00000000.jpg                0        512   
1  ./Bottle Images/images_raw  00000000.jpg                0        512   
2  ./Bottle Images/images_raw  00000001.jpg                1        512   
3  ./Bottle Images/images_raw  00000001.jpg                1        512   
4  ./Bottle Images/images_raw  00000001.jpg                1        512   

   img_height img_depth  ann_bbox_xmin  ann_bbox_ymin  ann_bbox_width  \
0         512                    370.5          214.5            47.0   
1         512                    348.5          147.5           124.0   
2         512                     84.5          100.5           191.0   
3         512                    263.5          172.5           114.0   
4         512                    309.5          206.5            53.0   

   ann_bbox_height  cat_id cat_name  
0            144.0       0   bottle  
1             90.0       0   bottl

## 🖼️ Visualisasi Anotasi untuk Verifikasi

Untuk memastikan proses konversi label berhasil, kita tampilkan beberapa gambar dengan anotasi yang telah diperbarui. Ini membantu memverifikasi bahwa anotasi sudah sesuai dan masih terhubung dengan gambar yang tepat.

Langkah ini sangat berguna untuk quality control sebelum melanjutkan ke pelabelan ulang dalam format YOLO.


In [18]:
import json
import pandas as pd
import os

# Path ke COCO JSON
path_to_annotations = r"./ImageClassesCombinedWithCOCOAnnotations/coco_instances.json"

# Load COCO JSON
with open(path_to_annotations, 'r') as f:
    coco = json.load(f)

# Ambil data images dan annotations
images = coco['images']
annotations = coco['annotations']
categories = {cat['id']: cat['name'] for cat in coco['categories']}

# Jumlah images
num_images = len(images)

# Buat list cat_name dari annotations (sebelum disatukan)
cat_names_original = [categories[ann['category_id']] for ann in annotations]

# Hitung class counts sebelum disatukan
df_orig = pd.DataFrame({'cat_name': cat_names_original})
class_counts_orig = df_orig['cat_name'].value_counts()

# Karena kamu ingin semua kelas disatukan menjadi 'bottle'
cat_names_new = ['bottle'] * len(annotations)  # semua jadi 'bottle'
df_new = pd.DataFrame({'cat_name': cat_names_new})
class_counts_new = df_new['cat_name'].value_counts()

# Output yang mirip dengan yang kamu mau
print(f"Number of images: {num_images}")
print(f"Number of classes (original): {len(class_counts_orig)}")
print(f"Classes (original): {list(class_counts_orig.index)}")
print(f"Class counts (original):\n{class_counts_orig}")
print("\n--- After merging all classes to 'bottle' ---")
print(f"Number of classes: {len(class_counts_new)}")
print(f"Classes: {list(class_counts_new.index)}")
print(f"Class counts:\n{class_counts_new}")
print(f"Path to annotations:\n{os.path.dirname(path_to_annotations)}")

Number of images: 15000
Number of classes (original): 5
Classes (original): ['plastic', 'beer', 'wine', 'soda', 'water']
Class counts (original):
cat_name
plastic    6050
beer       6047
wine       5990
soda       5946
water      5940
Name: count, dtype: int64

--- After merging all classes to 'bottle' ---
Number of classes: 1
Classes: ['bottle']
Class counts:
cat_name
bottle    29973
Name: count, dtype: int64
Path to annotations:
./ImageClassesCombinedWithCOCOAnnotations


In [20]:
import json
import copy

# Load COCO JSON
with open(path_to_annotations, 'r') as f:
    coco = json.load(f)

# Buat salinan data
coco_new = copy.deepcopy(coco)

# Ganti categories jadi hanya 1 kelas "bottle" dengan id=0 (atau id=1 sesuai kebutuhan YOLO)
coco_new['categories'] = [{"id": 0, "name": "bottle", "supercategory": "bottle_categories"}]

# Ganti semua category_id di annotations jadi 0
for ann in coco_new['annotations']:
    ann['category_id'] = 0

# Simpan ke file baru
new_json_path = "./ImageClassesWtihCoco/coco_instances_bottle.json"
with open(new_json_path, 'w') as f:
    json.dump(coco_new, f)

print(f"File anotasi baru disimpan di: {new_json_path}")

File anotasi baru disimpan di: ./ImageClassesWtihCoco/coco_instances_bottle.json


## 🧹 Membersihkan dan Menyiapkan Folder Dataset

Sebelum mengekspor dataset ke format YOLO, kita bersihkan terlebih dahulu struktur folder output agar tidak tercampur dengan file lama. Kemudian buat ulang struktur direktori seperti `images/train`, `images/val`, `labels/train`, `labels/val`.

Langkah ini penting untuk memastikan struktur dataset sesuai standar YOLOv8 dan tidak terjadi konflik atau file sisa dari eksperimen sebelumnya.


In [26]:
import os

def get_folder_filetypes(root_folder):
    for foldername, subfolders, filenames in os.walk(root_folder):
        # Ambil ekstensi file unik di folder ini
        file_extensions = set()
        for f in filenames:
            ext = os.path.splitext(f)[1].lower()  # ekstensi termasuk titik, contoh '.jpg'
            if ext:
                file_extensions.add(ext)
            else:
                file_extensions.add('(no extension)')
        print(f"Folder: {foldername}")
        if file_extensions:
            print(f"  File types: {', '.join(sorted(file_extensions))}")
        else:
            print("  File types: (empty folder)")
        print()

# Contoh pakai:
root_dir = './ImageClassesWtihCoco'  # ganti sesuai folder yang mau dicek
get_folder_filetypes(root_dir)

Folder: ./ImageClassesWtihCoco
  File types: .json, .yaml

Folder: ./ImageClassesWtihCoco\images_raw
  File types: .jpg

Folder: ./ImageClassesWtihCoco\yolo_labels
  File types: .txt



In [27]:
import os

images_dir = './ImageClassesWtihCoco/images_raw'
labels_dir = './ImageClassesWtihCoco/yolo_labels'

image_files = set(f for f in os.listdir(images_dir) if f.lower().endswith('.jpg'))
label_files = set(f for f in os.listdir(labels_dir) if f.lower().endswith('.txt'))

# Ubah ekstensi file label ke .jpg supaya bisa dibandingkan nama file
label_files_as_images = set(f.replace('.txt', '.jpg') for f in label_files)

# Cek gambar yang tidak punya label
images_without_label = image_files - label_files_as_images

# Cek label yang tidak punya gambar
labels_without_image = label_files_as_images - image_files

print(f"Total images: {len(image_files)}")
print(f"Total labels: {len(label_files)}")

if len(images_without_label) == 0:
    print("✅ Semua gambar punya label yang sesuai.")
else:
    print(f"⚠️ Gambar tanpa label: {sorted(images_without_label)}")

if len(labels_without_image) == 0:
    print("✅ Semua label punya gambar yang sesuai.")
else:
    print(f"⚠️ Label tanpa gambar: {sorted(labels_without_image)}")

Total images: 15000
Total labels: 15000
✅ Semua gambar punya label yang sesuai.
✅ Semua label punya gambar yang sesuai.


------------
# Split Data
------------
## 🔀 Split Dataset Menjadi Train dan Validation

Untuk melatih model secara efektif, dataset perlu dibagi menjadi dua bagian:
- **Train set**: digunakan untuk melatih model.
- **Validation set**: digunakan untuk mengevaluasi performa model pada data yang belum pernah dilihat.

Pembagian dilakukan secara acak (dengan `random.seed` untuk reprodusibilitas) menggunakan rasio 80% untuk train dan 20% untuk validation.


In [32]:
import os
import shutil
import random
from tqdm import tqdm

# Paths
src_images = "./ImageClassesWtihCoco/images_raw"
src_labels = "./ImageClassesWtihCoco/yolo_labels"
dst_images_train = "./ImageClassesWtihCoco/images/train"
dst_images_val = "./ImageClassesWtihCoco/images/val"
dst_labels_train = "./ImageClassesWtihCoco/labels/train"
dst_labels_val = "./ImageClassesWtihCoco/labels/val"

# Buat folder output
for folder in [dst_images_train, dst_images_val, dst_labels_train, dst_labels_val]:
    os.makedirs(folder, exist_ok=True)

# Ambil semua nama file gambar
image_files = [f for f in os.listdir(src_images) if f.endswith(".jpg")]
random.shuffle(image_files)

# Split
split_ratio = 0.8
split_idx = int(len(image_files) * split_ratio)
train_files = image_files[:split_idx]
val_files = image_files[split_idx:]

# Copy file ke folder masing-masing
def move_files(file_list, dst_img_dir, dst_lbl_dir):
    for file in tqdm(file_list):
        base = os.path.splitext(file)[0]
        src_img = os.path.join(src_images, file)
        src_lbl = os.path.join(src_labels, base + ".txt")

        dst_img = os.path.join(dst_img_dir, file)
        dst_lbl = os.path.join(dst_lbl_dir, base + ".txt")

        shutil.copy2(src_img, dst_img)
        shutil.copy2(src_lbl, dst_lbl)

move_files(train_files, dst_images_train, dst_labels_train)
move_files(val_files, dst_images_val, dst_labels_val)

print("✅ Data split completed.")
print(f"Train: {len(train_files)} images")
print(f"Val: {len(val_files)} images")

100%|█████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:03<00:00, 792.32it/s]

✅ Data split completed.
Train: 12000 images
Val: 3000 images


-----------------
# Training YOLOv8 Ultralytics
-----------------

---

## ✅ **Jenis Model YOLOv8 dari Ultralytics**

YOLOv8 memiliki beberapa varian utama berdasarkan ukuran dan kompleksitas:

| Model        | Ukuran | Kecepatan ⚡   | Akurasi 🎯   | Kebutuhan Memori 💾 |
| ------------ | ------ | ------------- | ------------ | ------------------- |
| `yolov8n.pt` | Nano   | Sangat cepat  | Cukup baik   | Paling ringan       |
| `yolov8s.pt` | Small  | Cepat         | Lebih baik   | Ringan              |
| `yolov8m.pt` | Medium | Sedang        | Bagus        | Sedang              |
| `yolov8l.pt` | Large  | Lebih lambat  | Sangat bagus | Besar               |
| `yolov8x.pt` | XLarge | Paling lambat | Terbaik      | Paling besar        |

---

## ⚙️ Melatih Model YOLOv8

Setelah dataset siap dalam format dan struktur yang sesuai, langkah selanjutnya adalah melatih model menggunakan YOLOv8 dari Ultralytics.

Beberapa parameter penting saat pelatihan:
- `data`: File YAML yang mendeskripsikan lokasi data dan kelas (hanya 1 yaitu 'bottle').
- `model`: Arsitektur model YOLO yang digunakan (misalnya `yolov8n.pt` untuk model kecil).
- `epochs`: Jumlah epoch pelatihan.
- `imgsz`: Ukuran input gambar.
- `batch`: Ukuran batch saat pelatihan.

Model akan disimpan secara otomatis di direktori `runs/detect/`.


In [37]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # Bisa diganti ke yolov8s.pt, yolov8m.pt, dst.
model.train(
    data="./ImageClassesWtihCoco/dataset.yaml", 
    epochs=50,             # jumlah epoch, sesuaikan kebutuhan
    imgsz=640,             # ukuran input gambar
    batch=16,              # batch size (sesuaikan dengan GPU/VRAM kamu)
    name='yolov8_bottle',  # nama folder output hasil training
    device='cpu'             # gunakan GPU 0, ganti sesuai device kamu
)

New https://pypi.org/project/ultralytics/8.3.141 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.134  Python-3.12.7 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=./ImageClassesWtihCoco/dataset.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8_bottle5, nbs=64, nms=Fa

train: Scanning C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\labels\train.cache... 12

train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000000.jpg: 2 duplicate labels removed
train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000002.jpg: 3 duplicate labels removed
train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000005.jpg: 2 duplicate labels removed
train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000009.jpg: 3 duplicate labels removed
train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000010.jpg: 1 duplicate labels removed
train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000012.jpg: 2 duplicate labels removed
train: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\train\00000013.jpg: 2 duplicate labels removed
train: C:\Users\Admi

val: Fast image access  (ping: 0.00.0 ms, read: 256.4129.7 MB/s, size: 43.3 KB)


C:\Users\Admin\Documents\Kode Python\Deteksi Objek\cnnyolo\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\labels\val.cache... 3000 i

...

                   all       3000       6037      0.994      0.981      0.995      0.951



50 epochs completed in 76.284 hours.
Optimizer stripped from runs\detect\yolov8_bottle5\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\yolov8_bottle5\weights\best.pt, 6.2MB

Validating runs\detect\yolov8_bottle5\weights\best.pt...
Ultralytics 8.3.134  Python-3.12.7 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 94/94 [06:35


                   all       3000       6037      0.997       0.98      0.995      0.951
Speed: 2.2ms preprocess, 121.5ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs\detect\yolov8_bottle5


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000017D299E2150>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480


---

## 📊 **Standar Penilaian Object Detection (YOLOv8 dan sejenisnya)**

### 🔹 1. **mAP (Mean Average Precision)**

Metrik utama untuk mengevaluasi model deteksi objek.

| Metrik            | Rentang Nilai | Keterangan                                                         |
| ----------------- | ------------- | ------------------------------------------------------------------ |
| **mAP\@0.5**      | < 0.50        | **Buruk** – Model gagal mendeteksi dengan baik.                    |
|                   | 0.50 – 0.70   | **Cukup** – Bisa digunakan, tapi masih perlu perbaikan.            |
|                   | 0.70 – 0.90   | **Bagus** – Model bekerja dengan cukup baik.                       |
|                   | > 0.90        | **Sangat Bagus** – Model sangat akurat. ✅                          |
| **mAP\@0.5:0.95** | < 0.30        | **Buruk** – Model tidak konsisten pada threshold IoU tinggi.       |
|                   | 0.30 – 0.60   | **Cukup** – Masih bisa dipakai, tapi kurang presisi.               |
|                   | 0.60 – 0.85   | **Bagus** – Model akurat di berbagai tingkat overlap.              |
|                   | > 0.85        | **Sangat Bagus** – Akurasi tinggi dan stabil di semua threshold. ✅ |

---

### 🔹 2. **Precision (P) dan Recall (R)**

| Metrik        | Rentang Nilai | Keterangan                                                        |
| ------------- | ------------- | ----------------------------------------------------------------- |
| **Precision** | < 0.50        | **Buruk** – Terlalu banyak false positive.                        |
|               | 0.50 – 0.80   | **Cukup** – Masih sering salah mengenali objek.                   |
|               | 0.80 – 0.95   | **Bagus** – Prediksi tepat dan relevan.                           |
|               | > 0.95        | **Sangat Bagus** – Prediksi sangat presisi. ✅                     |
| **Recall**    | < 0.50        | **Buruk** – Banyak objek tidak terdeteksi.                        |
|               | 0.50 – 0.80   | **Cukup** – Masih banyak objek terlewat.                          |
|               | 0.80 – 0.95   | **Bagus** – Hampir semua objek berhasil ditemukan.                |
|               | > 0.95        | **Sangat Bagus** – Hampir semua objek dideteksi tanpa terlewat. ✅ |

---

### 🔹 3. **Loss (Semakin Rendah Semakin Baik)**

| Jenis Loss   | Rentang Baik | Catatan Penting                                     |
| ------------ | ------------ | --------------------------------------------------- |
| **Box Loss** | < 0.5        | Bounding box akurat                                 |
| **Cls Loss** | < 0.3        | Klasifikasi objek tepat                             |
| **DFL Loss** | < 1.0        | Akurasi prediksi lokasi bbox dengan distribusi baik |

> ⚠️ **Catatan**: Tidak ada angka “mutlak”, karena nilai loss tergantung dataset, kompleksitas objek, dan ukuran input. Yang terpenting adalah **tren penurunan dari waktu ke waktu.**

---

### 🔹 4. **Patokan Umum Model yang “Siap Dipakai”**
Metode ini akan menghasilkan:
- **mAP (mean Average Precision)**: Ukuran utama untuk performa deteksi objek.
- **Confusion Matrix**
- **Precision/Recall Curve**

  
Model dianggap **siap digunakan** secara umum jika:

* `mAP@0.5 ≥ 0.90`
* `mAP@0.5:0.95 ≥ 0.75`
* `Precision ≥ 0.90`
* `Recall ≥ 0.90`
* Loss (Box, Cls, DFL) **stabil menurun**

---

In [38]:
from ultralytics import YOLO

# Load model hasil training (misal best.pt)
model = YOLO('runs/detect/yolov8_bottle5/weights/best.pt')

# Lakukan evaluasi pada validation set
metrics = model.val()

# Tampilkan metrik evaluasi
print(f"mAP@0.5-0.95: {metrics.box.map:.4f}")
print(f"mAP@0.5     : {metrics.box.map50:.4f}")
print(f"mAP@0.75    : {metrics.box.map75:.4f}")
print(f"Mean Precision (MP): {metrics.box.mp:.4f}")
print(f"Mean Recall (MR)   : {metrics.box.mr:.4f}")
print(f"F1 Score (avg)     : {metrics.box.f1.mean():.4f}")

Ultralytics 8.3.134  Python-3.12.7 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 268.549.6 MB/s, size: 33.3 KB)


val: Scanning C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\labels\val.cache... 3000 i

val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000001.jpg: 3 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000003.jpg: 2 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000004.jpg: 2 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000006.jpg: 3 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000007.jpg: 3 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000008.jpg: 2 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteksi Objek\Bottle\ImageClassesWtihCoco\images\val\00000011.jpg: 3 duplicate labels removed
val: C:\Users\Admin\Documents\Kode Python\Deteks


C:\Users\Admin\Documents\Kode Python\Deteksi Objek\cnnyolo\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 188/188 [05:


                   all       3000       6037      0.997       0.98      0.995      0.951
Speed: 1.6ms preprocess, 106.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\val
mAP@0.5-0.95: 0.9508
mAP@0.5     : 0.9947
mAP@0.75    : 0.9874
Mean Precision (MP): 0.9969
Mean Recall (MR)   : 0.9803
F1 Score (avg)     : 0.9885


## 💾 Ekspor Model ke Format Lain

Model YOLOv8 yang sudah terlatih dapat diekspor ke berbagai format agar bisa digunakan di berbagai platform:

- `torchscript` → PyTorch
- `onnx` → Untuk deployment cross-platform
- `openvino`, `tflite`, `coreml` → Untuk perangkat edge seperti Intel, Android, iOS
- `engine` → TensorRT untuk akselerasi GPU

In [43]:
# Save Model ONNX
from ultralytics import YOLO

# Gunakan path yang benar
model = YOLO("runs/detect/yolov8_bottle5/weights/best.pt")

# Export ke format ONNX
model.export(format="onnx")

Ultralytics 8.3.134  Python-3.12.7 torch-2.7.0+cpu CPU (11th Gen Intel Core(TM) i3-1115G4 3.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\yolov8_bottle5\weights\best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 5, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.46', 'onnxruntime'] not found, attempting AutoUpdate...
Defaulting to user installation because normal site-packages is not writeable

requirements: AutoUpdate success  1.6s, installed 3 packages: ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.46', 'onnxruntime']
WARNING requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.18.0 opset 19...
WARNING ONNX: simplifier failure: No module named 'onnxslim'
ONNX: export success  2.9s, saved as 'runs\detect\yolov8_bottle5\weights\best.onnx' (11.7 MB)

Export complete (3.3s)
Results saved to C:\U

'runs\\detect\\yolov8_bottle5\\weights\\best.onnx'

In [47]:
import onnx
from onnx import helper

model = onnx.load("runs/detect/yolov8_bottle5/weights/best.onnx")
onnx.checker.check_model(model)
print("Model ONNX valid!")

Model ONNX valid!


In [55]:
import subprocess
import time
from IPython.display import IFrame, display

# Jalankan netron via python -m netron
proc = subprocess.Popen(['python', '-m', 'netron', 'runs/detect/yolov8_bottle5/weights/best.onnx', '--port', '8081'])

time.sleep(3)  # tunggu server netron siap

display(IFrame(src='http://localhost:8080', width=900, height=600))

# Nanti kalau mau matikan server:
# proc.terminate()